In [1]:
import cvxopt as cvx
from cvxopt import blas, solvers
import cvxpy as cp

import scipy.optimize as sco

import numpy as np
import pandas as pd

In [2]:
def port_ret(weights, ret, risk_free = 0):
    #needs to be array
    port_ret = risk_free + weights.dot(ret - risk_free)
    
    return(port_ret)

In [3]:
def port_var(weights, cov):
    var = weights.dot(cov).dot(weights)
    return(var)

In [4]:
def kelly_objective(weights,ret, cov, risk_free = 0):
    kelly_ret = port_ret(weights, ret)
    var = port_var(weights, cov)
    
    #obj = (-sum(weights) * var) + kelly_ret -risk_free
    
    obj = -(risk_free + sum(weights)*(kelly_ret - risk_free) -((sum(weights)**2)*var)/(2))
    return(obj)

In [88]:
def kelly_cri(ret, cov ,risk_free = 0):
    
    #need smaller step size
    num_assets = ret.shape[0]
    args = (ret, cov,risk_free)
    constraints = ({'type':'ineq', 'fun': lambda x: x},#all elements greater than one
                  {'type':'ineq', 'fun': lambda x: 1 - np.sum(x)}  )  # sum <= 1
    
    result = sco.minimize(kelly_objective, num_assets*[1./num_assets,], args=args, 
                          method='SLSQP', constraints=constraints) 
    
    return (result)

In [177]:
def uncons_kelly(cov, ret, risk_free_rate = 0):
    
    if np.linalg.det(cov) == 0:
        return (print("Singular Matrix"))
    else:
        kelly = np.linalg.inv(cov).dot((ret-risk_free_rate))
        return kelly

In [178]:
def theoretical_kelly_lev(mu_1, mu_2, s_1, s_2, corr):
    
    weight_1 = (mu_1 -mu_2 +(s_2 + corr * s_1)*s_2)/(s_1**2+s_2**2 - 2*corr*s_1*s_2)
    weight_2 = (mu_2 -mu_1 +(s_1 + corr * s_2)*s_1)/(s_1**2+s_2**2 - 2*corr*s_1*s_2)
    
    cash = 1- weight_1 - weight_2
    
    return (weight_1, weight_2, cash)

In [215]:
def get_cov(std_1, std_2, corr):
    cov_1_2 = std_1 * std_2/100
    cov = np.array([[std_1**2/100 ,corr * cov_1_2],[corr * cov_1_2,std_2**2/100]])
    
    return cov

# Kelly Criterion Continous Distribution

TODO:


In [216]:
rf_rate = 0
ret = np.array([7,10])
corr = 0
std_1 = 20
std_2 = 30

cov = get_cov(std_1, std_2, corr)

## Original Kelly

The optimal Weights for a portolio (the one with the highest geometric growth) is given by taking the derivative og the portfolio growth and setting it to zero. The portfolio growth is

 $g_\infty = r + F^T(M - r) - \frac{F^T C F}{2}$

Where $r$ is the riskless rate, $F$ is the vector of weights, $M$ is the mean return vector and $C$ is the covariance vector.

To get the optimal weights one take the derivative of the growth rate, $g_\infty$, with respect to the weight, $F$, and minimize it. The derivative is given below

$\frac{\partial g_\infty}{\partial F} = C \dot F + M - r $

With no constraints the optimal weight $F^*$ is given by

$F^* = C^{-1} M-r$



In [217]:
uncons_kelly(cov, ret, risk_free_rate = rf_rate)

array([1.75      , 1.11111111])

## Fractional Betting and Properties

When considering the portfolio as a continous process with kelly weights one can calculate some properties for the portfolio, such as time until a certain goal, probability of a drawdown and similar. 

In [218]:
print("Return for Full Kelly: " + str(port_ret(uncons_kelly(cov, ret, risk_free_rate = 0), ret, risk_free = 0)) + "%")
print("Volatility for Full Kelly: " + str(np.sqrt(port_var(uncons_kelly(cov, ret, risk_free_rate = 0), cov)).round(2)) + "%")



Return for Full Kelly: 23.36111111111111%
Volatility for Full Kelly: 4.83%


## Theoretical Kelly for two assets: Unleverd and no short selling


If one have two assets where $a_1, a_2$ are the percentage gain for assets 1 and 2, also $b_1, b_2$ are the percentage loss for the same. $\rho$ is the correlation of the assets and $p$ is the possability of comovments which is defined as $p = \frac{\rho + 1}{2}$.


If one does a taylor expansion of the geometric growth one will find that the geometric growth, $g = \mu - \frac{\sigma^2}{2}$ where $\mu = \frac{a-b}{2}$ and $\sigma^2 = \frac{a+b}{2}$. This is the same as for the continous case. 

The optimal weight for an asset, say asset 1, is calculated as $ w_A = \frac{\mu_1-\mu_2 + (\sigma_2 -\rho\sigma_1)\sigma_2}{\sigma_1^2 + \sigma_2^2 - 2\rho\sigma_1\sigma_2}$

If $w_1 + w_2 < 1$, then the rest of the bankroll is held in cash. 

CAUTION: This formula only works for two assets. The formula also has the implicit constrants of no leverage and no short selling.

In [223]:
def theoretical_kelly_lev(mu_1, mu_2, s_1, s_2, corr, rf_rate):
    mu_1 = mu_2 - rf_rate
    mu_2 = mu_2 - rf_rate
    
    
    weight_1 = (mu_1 -mu_2 +(s_2 - corr * s_1)*s_2)/(s_1**2+s_2**2 - 2*corr*s_1*s_2)
    weight_2 = (mu_2 -mu_1 +(s_1 - corr * s_2)*s_1)/(s_1**2+s_2**2 - 2*corr*s_1*s_2)
    
    cash = 1- weight_1 - weight_2
    
    return (weight_1, weight_2, cash)

In [224]:
theoretical_kelly_lev(ret[0], ret[1], std_1, std_2, corr, rf_rate)

(0.6923076923076923, 0.3076923076923077, 0.0)

## Constrained Kelly

When there is contraints on the portfolio (short selling, leverage, and so on) one can calculate the portfolio return for the portfolio and use quadratic maximazation to find the optimal weights under said constrinats. The portfolio porperties are:

$g_\infty = F^T M$

and

$\sigma^2 = F^T C F $

and since for the single asset case, the growth rate is

$g_\infty = r + f(m-r) - \frac{f^2 s^2}{2}$


In the constrianed case one might not reach the optimal point, so one cannot take the derivative and set to zero. Instead one has to maximize the return of the portfolio under certain constriants (short selling, leverage, and so on). 


In [225]:
result = kelly_cri(ret, cov, risk_free = 0)

In [226]:
result['x'].round(2)

array([0.46, 0.54])